In [10]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

# 확인 (테스트용)
# print(f"OpenAI API Key: {openai_api_key}")
# print(f"Anthropic API Key: {anthropic_api_key}")
# print(f"Gemini API Key: {gemini_api_key}")

In [3]:
from langchain_openai import ChatOpenAI
import openai

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

openai_models = openai.Model.list()
print(openai_models)

# # 질의내용
# question = "대한민국의 수도는 어디인가요?"

# # 질의
# print(f"[답변]: {llm.invoke(question)}")


APIRemovedInV1: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


블로그
https://velog.io/@woody_ahn/LangChain%EA%B3%BC-GPT-4o-mini%EB%A1%9C-%EA%B5%AC%ED%98%84%ED%95%98%EB%8A%94-%EA%B0%84%EB%8B%A8%ED%95%9C-%ED%8E%98%EB%A5%B4%EC%86%8C%EB%82%98-%EC%B1%97%EB%B4%87

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [3]:
# initialize language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# memory key
HISTORY_MEMORY_KEY = "chat_history"

# initialize memory for chat history
memory = ConversationBufferMemory(
    memory_key=HISTORY_MEMORY_KEY,
    return_messages=True,
)
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter(HISTORY_MEMORY_KEY)
)

/tmp/ipykernel_30953/3690843352.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


NameError: name 'itemgetter' is not defined

ConversationBufferMemory: 모든 대화 유지
ConversationBufferWindowMemory: 최근 K개의 대화만을 유지.
ConversationTokenBufferMemory: 토큰 길이로 대화 내용 유지.
ConversationEntityMemory: 대화에서 특정 entity에 대한 주어진 사실을 기억. 대화에서 entity에 대한 정보를 추출하고 해당 entity에 대한 지식을 축적함.
ConversationKGMemory: 지식 그래프를 사용한다고 함.
ConversationSummaryMemory: 대화가 진행되는 동안 대화를 요약하여 저장.
ConversationSummaryBufferMemory: 이전 대화는 요약해서 저장하고, 최근 대화 내용도 유지
VectorStoreRetrieverMemory: 메모리로 벡터 DB를 사용. 상위 K개 쿼리.

In [ ]:
# create prompt for persona chat
system_prompts = "".join(
    [
        "너의 이름은 민지.",
		"취미: 영화보기, 요리하기, 쇼핑하기",
        "너는 대화할 때 이모티콘을 자주 써.",
    ]
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompts),
        MessagesPlaceholder(variable_name=HISTORY_MEMORY_KEY),
        ("human", "{question}"),
    ]
)

langchain 공식페이지

In [8]:
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableWithMessageHistory

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

# 메모리 객체 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ConversationChain 대체: RunnableWithMessageHistory 사용
conversation = RunnableWithMessageHistory(
    llm=llm,
    memory=memory
)


TypeError: RunnableWithMessageHistory.__init__() missing 2 required positional arguments: 'runnable' and 'get_session_history'

In [6]:
# 대화 시작
# 초기 프롬프트 설정
initial_prompt = SystemMessage(
    content="""You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until the current one is fully addressed or skipped. 
You may ask up to 2 follow-up questions for clarification or more detail after the user's response to each main question. Once 2 follow-ups are reached, automatically proceed to the next main question, even if the user continues elaborating. Before moving to the next question, always provide a brief response acknowledging or reacting to the user’s previous answer.

Rules:
1. Strictly limit follow-up questions to a maximum of 2 per main question.
2. If the user provides a positive or neutral answer but lacks detail, ask follow-ups to encourage more detail or exploration of their thoughts.
3. If the user provides an elaborate or comprehensive answer, skip follow-ups and proceed to the next question after providing a brief reaction.
4. If the user skips a question or is uninterested, acknowledge this and move to the next question.
5. All responses must be in Korean.
6. Always provide a brief reaction (e.g., appreciation, empathy, or encouragement) to the user's answer before moving to the next question.
7. After all four main questions are asked, conclude the conversation with a polite remark in Korean.

At the start of the conversation, introduce the purpose with: "마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요."

Main Questions:
1. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?
2. 현재 관심 있는 취미나 활동이 있으신가요? 여가 시간에 주로 무엇을 하시나요?
3. 중요하게 생각하는 개인적인 가치나 목표가 있으신가요? 예를 들어, 가족, 커리어, 자기 개발 등 어떤 것들이 있나요?
4. 보통 스트레스를 어떻게 관리하시나요? 휴식, 운동, 취미 활동 등 어떤 방법을 사용하시나요?

Follow-up Question Guidelines:
1. If the user gives a neutral or positive response but lacks detail, ask follow-ups to explore further (e.g., "그럼 구체적으로 어떤 활동을 해보고 싶으세요?" or "그 이유를 좀 더 들려주실 수 있나요?").
2. If the user explicitly declines to elaborate or expresses disinterest, skip follow-ups and proceed to the next main question with a brief acknowledgment.
3. Ensure follow-ups are friendly and encourage conversation without pressuring the user.

Termination: If the user explicitly asks to stop, politely end the conversation in Korean."""
)

# 첫 질문 메시지 설정
human_message = HumanMessage(content="안녕하세요, 질문을 시작해주세요.")

# 대화 시작
response = conversation.invoke([initial_prompt, human_message])

# 모델 응답 출력
print(response.content)

# 대화 진행
while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() in ["그만", "종료", "quit", "exit"]:
        print("대화를 종료합니다. 감사합니다!")
        break

    # 모델에 사용자 입력 전달
    response = conversation.invoke([HumanMessage(content=user_input)])
    print(f"[응답]: {response.content}")

NameError: name 'conversation' is not defined

In [15]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import List
from langchain_core.messages import BaseMessage, AIMessage
from pydantic import BaseModel, Field

# InMemoryHistory 클래스 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Clear all messages."""
        self.messages = []

# 세션별 기록 관리 함수
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

# OpenAI LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# RunnableWithMessageHistory 초기화
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history,
)

# 초기 메시지 설정
initial_prompt = SystemMessage(content="You are a helpful assistant.")
user_message = HumanMessage(content="안녕하세요, 질문을 시작해주세요.")

# 대화 시작 (messages 리스트로 전달)
response = conversation.invoke(
    [initial_prompt, user_message],  # 메시지 객체의 리스트
    config={"configurable": {"session_id": "session_1"}}
)

# 결과 출력
print(response)


content='안녕하세요! 어떤 질문이든지 말씀해 주세요. 도움이 필요하신 부분이 있다면 최선을 다해 도와드리겠습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 25, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None} id='run-f74f9f58-6512-4e66-a1a8-3b724e251cab-0' usage_metadata={'input_tokens': 25, 'output_tokens': 31, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import List
from langchain_core.messages import BaseMessage, AIMessage
from pydantic import BaseModel, Field

# InMemoryHistory 클래스 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Clear all messages."""
        self.messages = []

    def get_recent_messages(self, max_messages: int) -> List[BaseMessage]:
        """Retrieve the most recent messages."""
        return self.messages[-max_messages:]  # 최근 max_messages개의 메시지만 반환

# 세션별 기록 관리 함수
store = {}

def get_session_history(session_id: str) -> InMemoryHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

# OpenAI LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# RunnableWithMessageHistory 초기화
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history,
)

# 초기 메시지 설정
initial_prompt = SystemMessage(content="""

                                """)
user_message = HumanMessage(content="안녕하세요, 질문을 시작해주세요.")

# 대화 시작
session_id = "session_1"
history = get_session_history(session_id)

# 최근 메시지 제한 (최대 n개)
max_recent_messages = 30

# 대화 기록 관리
# 초기 메시지 추가
if not history.messages:  # history에 메시지가 없는 경우에만 추가
    history.add_messages([initial_prompt, user_message])

recent_messages = history.get_recent_messages(max_recent_messages)  # 최근 메시지 가져오기

# 모델 호출
response = conversation.invoke(
    recent_messages,  # 최근 메시지 전달
    config={"configurable": {"session_id": session_id}}
)

# 결과 출력
print(response)

# 대화 진행
while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() in ["그만", "종료", "quit", "exit"]:  # 종료 조건
        print("대화를 종료합니다. 감사합니다!")
        break

    # 새로운 메시지 추가
    new_message = HumanMessage(content=user_input)
    history.add_messages([new_message])

    # 최근 메시지 가져오기
    recent_messages = history.get_recent_messages(max_recent_messages)

    # 전달된 메시지 출력
    print("======모델에 전달된 대화 내역:======")
    for msg in recent_messages:
        print(f"{msg.__class__.__name__}: {msg.content}")
    print('='*10)

    # 모델 호출
    response = conversation.invoke(
        recent_messages,
        config={"configurable": {"session_id": session_id}}
    )

    # 응답 출력
    print(f"[응답]: {response.content}")

content='안녕하세요! 어떤 질문이든지 말씀해 주세요. 도와드릴 준비가 되어 있습니다!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 47, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'stop', 'logprobs': None} id='run-686134c9-f8cb-4355-a6e6-b78a5cbeb863-0' usage_metadata={'input_tokens': 47, 'output_tokens': 22, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
======모델에 전달된 대화 내역:======
SystemMessage: You are a helpful assistant.
HumanMessage: 안녕하세요, 질문을 시작해주세요.
SystemMessage: You are a helpful assistant.
HumanMessage: 안녕하세요, 질문을 시작해주세요.
AIMessage: 안녕하세요! 어떤 질문이든지 말씀해 주세요. 도와드릴 준비가 되어 있습니다!
HumanMessage: 

In [8]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import List
from langchain_core.messages import BaseMessage, AIMessage
from pydantic import BaseModel, Field

# InMemoryHistory 클래스 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Clear all messages."""
        self.messages = []

    def get_recent_messages(self, max_messages: int) -> List[BaseMessage]:
        """Retrieve the most recent messages."""
        return self.messages[-max_messages:]  # 최근 max_messages개의 메시지만 반환

# 세션별 기록 관리 함수
store = {}

def get_session_history(session_id: str) -> InMemoryHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

# OpenAI LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# RunnableWithMessageHistory 초기화
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history,
)

# 초기 메시지 설정
initial_prompt = SystemMessage(content=
"""You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until the current one is fully addressed or skipped. 
You may ask up to 2 follow-up questions for clarification or more detail after the user's response to each main question. Once 2 follow-ups are reached, automatically proceed to the next main question, even if the user continues elaborating. Before moving to the next question, always provide a brief response acknowledging or reacting to the user’s previous answer.

Rules:
1. Strictly limit follow-up questions to a maximum of 2 per main question.
2. If the user provides a positive or neutral answer that is not overly elaborate, ask follow-up questions to encourage more detail or exploration of their thoughts.
3. If the user provides an elaborate or comprehensive answer, skip follow-ups and proceed to the next question after providing a brief reaction.
4. If the user skips a question or is uninterested, acknowledge this and move to the next question.
5. All responses must be in Korean.
6. Always provide a brief reaction (e.g., appreciation, empathy, or encouragement) to the user's answer before moving to the next question.
7. After all four main questions are asked, conclude the conversation with a polite remark in Korean.

At the start of the conversation, introduce the purpose with: "마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요."

Main Questions:
1. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?
2. 현재 관심 있는 취미나 활동이 있으신가요? 여가 시간에 주로 무엇을 하시나요?
3. 중요하게 생각하는 개인적인 가치나 목표가 있으신가요? 예를 들어, 가족, 커리어, 자기 개발 등 어떤 것들이 있나요?
4. 보통 스트레스를 어떻게 관리하시나요? 휴식, 운동, 취미 활동 등 어떤 방법을 사용하시나요?

Follow-up Question Guidelines:
1. If the user gives a neutral or positive response but lacks detail, ask follow-ups to explore further (e.g., "그럼 구체적으로 어떤 활동을 해보고 싶으세요?" or "그 이유를 좀 더 들려주실 수 있나요?").
2. If the user explicitly declines to elaborate or expresses disinterest, skip follow-ups and proceed to the next main question with a brief acknowledgment.
3. Ensure follow-ups are friendly and encourage conversation without pressuring the user.

Termination: If the user explicitly asks to stop, politely end the conversation in Korean.
"""
                                )
user_message = HumanMessage(content="안녕하세요, 질문을 시작해주세요.")

# 대화 시작
session_id = "session_1"
history = get_session_history(session_id)

# 최근 메시지 제한 (최대 n개)
max_recent_messages = 30

# 대화 기록 관리
# 초기 메시지 추가
if not history.messages:  # history에 메시지가 없는 경우에만 추가
    history.add_messages([initial_prompt, user_message])

recent_messages = history.get_recent_messages(max_recent_messages)  # 최근 메시지 가져오기

# 모델 호출
response = conversation.invoke(
    recent_messages,  # 최근 메시지 전달
    config={"configurable": {"session_id": session_id}}
)

# 결과 출력
print(response)

# 대화 진행
while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() in ["그만", "종료", "quit", "exit"]:  # 종료 조건
        print("대화를 종료합니다. 감사합니다!")
        break

    # 새로운 메시지 추가
    new_message = HumanMessage(content=user_input)
    history.add_messages([new_message])

    # 최근 메시지 가져오기
    recent_messages = history.get_recent_messages(max_recent_messages)

    # 전달된 메시지 출력
    print("======모델에 전달된 대화 내역:======")
    for msg in recent_messages:
        print(f"{msg.__class__.__name__}: {msg.content}")
    print('='*10)

    # 모델 호출
    response = conversation.invoke(
        recent_messages,
        config={"configurable": {"session_id": session_id}}
    )

    # 응답 출력
    print(f"[응답]: {response.content}", flush=True)

content='안녕하세요! 마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요. 첫 번째 질문입니다. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 1137, 'total_tokens': 1200, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'stop', 'logprobs': None} id='run-98c19f0a-1afd-4207-8c75-6973f0cc86e9-0' usage_metadata={'input_tokens': 1137, 'output_tokens': 63, 'total_tokens': 1200, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
======모델에 전달된 대화 내역:======
SystemMessage: You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until 

In [9]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import List
from langchain_core.messages import BaseMessage, AIMessage
from pydantic import BaseModel, Field

# InMemoryHistory 클래스 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        """Clear all messages."""
        self.messages = []

    def get_recent_messages(self, max_messages: int) -> List[BaseMessage]:
        """Retrieve the most recent messages."""
        return self.messages[-max_messages:]  # 최근 max_messages개의 메시지만 반환

# 세션별 기록 관리 함수
store = {}

def get_session_history(session_id: str) -> InMemoryHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

# OpenAI LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# RunnableWithMessageHistory 초기화
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history,
)

# 초기 메시지 설정
initial_prompt = SystemMessage(content=
"""
You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until the current one is fully addressed or skipped.  
You may ask up to 2 follow-up questions for clarification or more detail after the user's response to each main question. Once 2 follow-ups are reached, automatically proceed to the next main question, even if the user continues elaborating. Before moving to the next question, always provide a brief response acknowledging or reacting to the user’s previous answer.

Rules:  
1. Strictly limit follow-up questions to a maximum of 2 per main question.  
2. If the user provides a positive or neutral answer that is not overly elaborate, ask follow-up questions to encourage more detail or exploration of their thoughts.  
3. If the user provides an elaborate or comprehensive answer, skip follow-ups and proceed to the next question after providing a brief reaction.  
4. If the user skips a question or is uninterested, acknowledge this and move to the next question.  
5. All responses must be in Korean.  
6. Always provide a brief reaction (e.g., appreciation, empathy, or encouragement) to the user's answer before moving to the next question.  
7. After all four main questions are asked, conclude the conversation with a polite remark in Korean.  

Introduction:  
At the start of the conversation, introduce the purpose with:  
"마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요."  

Main Questions:  
1. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?  
2. 현재 관심 있는 취미나 활동이 있으신가요? 여가 시간에 주로 무엇을 하시나요?  
3. 중요하게 생각하는 개인적인 가치나 목표가 있으신가요? 예를 들어, 가족, 커리어, 자기 개발 등 어떤 것들이 있나요?  
4. 보통 스트레스를 어떻게 관리하시나요? 휴식, 운동, 취미 활동 등 어떤 방법을 사용하시나요?  

Follow-up Question Guidelines:  
1. If the user gives a neutral or positive response but lacks detail, ask follow-ups to explore further (e.g., "그럼 구체적으로 어떤 활동을 해보고 싶으세요?" or "그 이유를 좀 더 들려주실 수 있나요?").  
2. If the user explicitly declines to elaborate or expresses disinterest, skip follow-ups and proceed to the next main question with a brief acknowledgment.  
3. Ensure follow-ups are friendly and encourage conversation without pressuring the user.  

Termination:  
If the user explicitly asks to stop, politely end the conversation in Korean.

Closing the Conversation:  
After all four main questions (and optional follow-ups) have been asked, conclude the conversation with:  
"감사합니다. 고객님과의 대화가 즐거웠습니다. 좋은 하루 보내세요!"
"""
                                )
user_message = HumanMessage(content="안녕하세요, 질문을 시작해주세요.")

# 대화 시작
session_id = "session_1"
history = get_session_history(session_id)

# 최근 메시지 제한 (최대 n개)
max_recent_messages = 30

# 대화 기록 관리
# 초기 메시지 추가
if not history.messages:  # history에 메시지가 없는 경우에만 추가
    history.add_messages([initial_prompt, user_message])

recent_messages = history.get_recent_messages(max_recent_messages)  # 최근 메시지 가져오기

# 모델 호출
response = conversation.invoke(
    recent_messages,  # 최근 메시지 전달
    config={"configurable": {"session_id": session_id}}
)

# 결과 출력
print(response)

# 대화 진행
while True:
    user_input = input("사용자 입력: ")
    if user_input.lower() in ["그만", "종료", "quit", "exit"]:  # 종료 조건
        print("대화를 종료합니다. 감사합니다!")
        break

    # 새로운 메시지 추가
    new_message = HumanMessage(content=user_input)
    history.add_messages([new_message])

    # 최근 메시지 가져오기
    recent_messages = history.get_recent_messages(max_recent_messages)

    # 전달된 메시지 출력
    print("======모델에 전달된 대화 내역:======")
    for msg in recent_messages:
        print(f"{msg.__class__.__name__}: {msg.content}")
    print('='*10)

    # 모델 호출
    response = conversation.invoke(
        recent_messages,
        config={"configurable": {"session_id": session_id}}
    )

    # 응답 출력
    print(f"[응답]: {response.content}", flush=True)
     # 마무리 인사 포함 여부 확인
    if "대화가 즐거웠습니다" in response.content:
        print("대화를 종료합니다. 감사합니다!")
        break

content='안녕하세요! 마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요. 첫 번째 질문입니다. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 1281, 'total_tokens': 1344, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'stop', 'logprobs': None} id='run-79ddcd87-d573-4453-ab0f-02ec3383965f-0' usage_metadata={'input_tokens': 1281, 'output_tokens': 63, 'total_tokens': 1344, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
======모델에 전달된 대화 내역:======
SystemMessage: 
You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until

KeyboardInterrupt: Interrupted by user

In [1]:
user_input3 = input("사용자 입력을 기다립니다: ")

In [5]:
### langchain 문서 참조 초기
# 프롬프트 설정
messages = [
    (
        "system",
        """You are a chatbot that asks the user four main questions, one at a time, and does not proceed to the next question until the current one is fully addressed or skipped. 
You may ask up to 2 follow-up questions for clarification or more detail after the user's response to each main question. Once 2 follow-ups are reached, automatically proceed to the next main question, even if the user continues elaborating. Before moving to the next question, always provide a brief response acknowledging or reacting to the user’s previous answer.

Rules:
1. Strictly limit follow-up questions to a maximum of 2 per main question.
2. If the user provides a positive or neutral answer that is not overly elaborate, ask follow-up questions to encourage more detail or exploration of their thoughts.
3. If the user provides an elaborate or comprehensive answer, skip follow-ups and proceed to the next question after providing a brief reaction.
4. If the user skips a question or is uninterested, acknowledge this and move to the next question.
5. All responses must be in Korean.
6. Always provide a brief reaction (e.g., appreciation, empathy, or encouragement) to the user's answer before moving to the next question.
7. After all four main questions are asked, conclude the conversation with a polite remark in Korean.

At the start of the conversation, introduce the purpose with: "마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요."

Main Questions:
1. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?
2. 현재 관심 있는 취미나 활동이 있으신가요? 여가 시간에 주로 무엇을 하시나요?
3. 중요하게 생각하는 개인적인 가치나 목표가 있으신가요? 예를 들어, 가족, 커리어, 자기 개발 등 어떤 것들이 있나요?
4. 보통 스트레스를 어떻게 관리하시나요? 휴식, 운동, 취미 활동 등 어떤 방법을 사용하시나요?

Follow-up Question Guidelines:
1. If the user gives a neutral or positive response but lacks detail, ask follow-ups to explore further (e.g., "그럼 구체적으로 어떤 활동을 해보고 싶으세요?" or "그 이유를 좀 더 들려주실 수 있나요?").
2. If the user explicitly declines to elaborate or expresses disinterest, skip follow-ups and proceed to the next main question with a brief acknowledgment.
3. Ensure follow-ups are friendly and encourage conversation without pressuring the user.

Termination: If the user explicitly asks to stop, politely end the conversation in Korean.
"""
    ),
    (
        "human",
        "안녕하세요, 질문을 시작해주세요."
    )
]

# 질문 시작
ai_msg = llm.invoke(messages)

# 결과 출력
print(ai_msg)

content='안녕하세요! 마지막으로 고객님을 조금 더 알아보고 싶어 몇 가지 질문을 드릴게요. 첫 번째 질문입니다. 새로운 언어를 배우려는 동기는 무엇인가요? 예를 들어, 여행, 직장, 개인적인 성장 등 어떤 이유가 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 570, 'total_tokens': 632, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-7aa69251-d423-4a27-9d17-4edc5d18168c-0' usage_metadata={'input_tokens': 570, 'output_tokens': 62, 'total_tokens': 632, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
